
# Creating and Using Indexes #

There are three types of Indexes in MySQL:
- **Primary key**
    - clustered
    - no duplicates
    - sorts data
- **Index** 
    - unclustered
    - no sorting of data
- **Unique** 
    - hybrid
        - clustered and sorts data if there is no primary key or no unique key
        - unclustered and no sorting otherwise
    - no duplicates

---
### Adding indexes to a pre-existing table ###
Format:  
`ALTER TABLE <table name> ADD`  
`[PRIMARY KEY|INDEX index_name|UNIQUE index_name] (<field name>,...<field name>)` 

### Removing indexes from a table ###
Format:  
`ALTER TABLE <table name> DROP [PRIMARY KEY|INDEX index_name]`  

### Showing existing indexes ###
`SHOW INDEXES FROM <table name>`

---
### Examples ###
The Papers table will be used as an example:

**Papers(narid, myid, year, title, author)**

Start with no indexes.

- add UNIQUE on year and narid  
`ALTER TABLE Papers ADD UNIQUE year_narid_uidx (year, narid)`  
Result: table records are sorted by year and then narid

- add INDEX on myid, produces no new sorting  
`ALTER TABLE Papers ADD INDEX myid_idx (myid)`  
Result: table records remain sorted by year and then narid

- add PRIMARY on myid, fails because myid has multiple duplicate entries   
`ALTER TABLE Papers ADD PRIMARY KEY (myid)` 

- add UNIQUE on narid  
`ALTER TABLE Papers ADD UNIQUE narid_uidx (narid)`  
Result: table records remain sorted by year and then narid

- drop year_narid_idx  
`ALTER TABLE Papers DROP INDEX year_narid_uidx`  
Result: table records resorted by narid according to remaining UNIQUE index

- add PRIMARY to (title, year), resorts table on PRIMARY  
`ALTER TABLE Papers ADD PRIMARY KEY (title,year)`  
Result: table records resorted by title and then year, according to new PRiMARY KEY

---

### Adding indexes in a CREATE TABLE statement ###
Formats:

`PRIMARY KEY (field name, ..., field name)`  
`UNIQUE index_name (field name..., field name)`  
`INDEX index_name (field name..., field name)`  

Example of adding indexes in a CREATE TABLE statement:

```sql
CREATE TABLE A (
  b integer,
  c integer,
  PRIMARY KEY (b),
  UNIQUE c_unique_idx (c)
) Engine = INNODB
```
---
## When indexes are used ##

An index is used to:

- find rows matching a WHERE clause
- select rows for a join
- sort the output
- do a search using LIKE when there is no initial wildcard character.
- look up data stored in the index without going into the data file.  This can be particularly important when using an unclustered index.  

---
## Frequent uses for indexes
- **Equality search** – when the query tries to find an exact match to a value.  For example in a join: 
```sql
	#match record values for fields with the same name
	FROM A JOIN B USING(cid) 

	#match record values for fields with different names
	FROM A JOIN B on aid = bid
```

- **Range query** – when the query is trying to find values in a range, for example, as part of a WHERE:

```sql
	#match a range of years
	WHERE 2010 <= year AND year <= 2014
``` 
- **Equality search** does equally well with a **clustered** or **unclustered index**.  
- **Range query** does best with a **clustered index**.
- **Range query** has satisfactory performance with an **unclustered index** if
	 - **the number of retrieved records is small**, or 
     - **only the index needs to be used to find the appropriate values** (instead of using the data records themselves)
---
# Using EXPLAIN to Understand Query Efficiency #

Using **EXPLAIN** before an SQL query **reveals information about how the query will be performed**.  This is very useful for making queries more efficient.

Example:  Here is the result of an EXPLAIN used before a simple query to the Papers table.

**Papers(narid, myid, year, title, author)**

We start with the **Papers table** with **no indexes**:

`mysql> explain select myid from Papers where year = '2010';`  

| id | select_type | table  | partitions | type | possible_keys | key  | key_len | ref  | rows | filtered | Extra       |
|----|-------------|--------|------------|------|---------------|------|---------|------|------|----------|-------------|
|  1 | SIMPLE      | Papers | NULL       | ALL  | NULL          | NULL | NULL    | NULL | 1687 |    10.00 | Using where |





**Columns** in the EXPLAIN Output (important ones are in **bold**)
- **id** – gives order of the tables and joins
- select_type – query type
- **table** – table name of the current join
- partitions – for partitioned tables (not used)
- **type** – join type
- possible keys – the potential indexes to use
- **key** – the actual index used
- key_len – number of fields in a multi-field key that are used
- ref – the field compared to the index used
- **rows** – an estimate of the rows to be examined in the current table
- filtered – an estimated percentage of rows left after they are examined 
- **extra** – other explanatory information about the query

Here's **:more information on the type (join type) field**:
- **type**:  (the join type from best to worst)
    - system – one row
    - const – one matching row
    - eq_ref – one row read from the current table for each combination of rows in the previous tables.  Requires a primary key or unique key index. 
    - ref – all rows with matching values are read from the current table for each combination of rows in the previous tables.  Requires an index.
    - ref or null – like ref, except a search for NULL values is added
    - range – a range of rows is read
    - index – the values are retrieved from the index
    - all – every row of the table is read


---
Look again at the explain result.

`mysql> explain select myid from Papers where year = '2010';`  

| id | select_type | table  | partitions | type | possible_keys | key  | key_len | ref  | rows | filtered | Extra       |
|----|-------------|--------|------------|------|---------------|------|---------|------|------|----------|-------------|
|  1 | SIMPLE      | Papers | NULL       | ALL  | NULL          | NULL | NULL    | NULL | 1687 |    10.00 | Using where |

The explain shows that 1687 rows (all the rows in the table) need to be examined to find the correct records.  This is a **heap scan**.  It estimates that approximately 10% of the records (filtered column) will have year = 2020.

Note that this **query is a range query**: find the papers with 2010 <= year <= 2010.  Range queries are best done with a primary key (clustered index), but an unclustered index will also work well if the number of records to retrieve is small (as it is here).

---

### Adding a primary key (clustered index) ###
Let's **add a primary key**.  We need to use two fields to the primary key because, obviously, year is not unique.


`#add primary key with two fields`  
`mysql> alter table Papers add primary key (year, title);`   
`mysql> show indexes from papers;`  

| Table  | Non_unique | Key_name | Seq_in_index | Column_name | Collation | Cardinality | Sub_part | Packed | Null | Index_type | Comment | Index_comment |
|--------|------------|----------|--------------|-------------|-----------|-------------|----------|--------|------|------------|---------|---------------|
| papers |          0 | PRIMARY  |            1 | year        | A         |          13 |     NULL | NULL   |      | BTREE      |         |               |
| papers |          0 | PRIMARY  |            2 | title       | A         |        1687 |     NULL | NULL   |      | BTREE      |         |               |

The table now has a **primary key** which is the concatenation of **two fields**.  The **year field comes first**.

`mysql> explain select myid from Papers where year = '2010';`  

| id | select_type | table  | partitions | type | possible_keys | key     | key_len | ref   | rows | filtered | Extra       |
|----|-------------|--------|------------|------|---------------|---------|---------|-------|------|----------|-------------|
|  1 | SIMPLE      | Papers | NULL       | ref  | PRIMARY       | PRIMARY | 1       | const |  162 |   100.00 | Using where |

**Explain** now shows that the query will only examine 162 rows and that the **primary key will be used**.

<span style="color:red">**By adding the index, the query is more efficient!**</span>

<span style="color:red">
**Important notes:**</span>    
- **<span style="color:red">Building an index takes longer than a query!</span>**
- **<span style="color:red">Indexes should only be used when a particular type of query occurs frequently!</span>**
    
**Suppose we had added the fields in reverse order.**

`#drop primary key and add back with fields in reverse order`    
`mysql> alter table papers drop primary key`  
`mysql> alter table Papers add primary key (title,year);`   
`mysql> show indexes from papers;`    

| Table  | Non_unique | Key_name | Seq_in_index | Column_name | Collation | Cardinality | Sub_part | Packed | Null | Index_type | Comment | Index_comment |
|--------|------------|----------|--------------|-------------|-----------|-------------|----------|--------|------|------------|---------|---------------|
| papers |          0 | PRIMARY  |            1 | title       | A         |        1606 |     NULL | NULL   |      | BTREE      |         |               |
| papers |          0 | PRIMARY  |            2 | year        | A         |        1687 |     NULL | NULL   |      | BTREE      |         |               |

The table again has a **primary key** which is the concatenation of two fields.  This time, the **title field comes first**.


`mysql> explain select myid from Papers where year = '2010';`  

| id | select_type | table  | partitions | type | possible_keys | key  | key_len | ref  | rows | filtered | Extra       |
|----|-------------|--------|------------|------|---------------|------|---------|------|------|----------|-------------|
|  1 | SIMPLE      | papers | NULL       | ALL  | NULL          | NULL | NULL    | NULL | 1687 |    10.00 | Using where |

As we can see, **having the title field come first doesn't help with the query**.  It has to again look at all the table rows.

<span style="color:red">**A compound key only helps if the search field comes first!**</span>

---
### Adding an unclustered index ###

Suppose the **table already has a primary key**.  To improve the query, we can **add a second index, but it has to be unclustered**.

`#drop primary key, add new narid primary key, add unclustered index for year`  
`mysql> alter table Papers drop primary key`    
`myslq> alter table Papers add primary key (narid)`  
`myslq> alter table Papers add index year_idx (year)`  
`mysql> show indexes from Papers;`      

| Table  | Non_unique | Key_name | Seq_in_index | Column_name | Collation | Cardinality | Sub_part | Packed | Null | Index_type | Comment | Index_comment |
|--------|------------|----------|--------------|-------------|-----------|-------------|----------|--------|------|------------|---------|---------------|
| papers |          0 | PRIMARY  |            1 | narid       | A         |        1687 |     NULL | NULL   |      | BTREE      |         |               |
| papers |          1 | year_idx |            1 | year        | A         |          13 |     NULL | NULL   |      | BTREE      |         |               |

The table now has a primary key on narid (clustered index) and an unclustered index on year.


`mysql> explain select myid from Papers where year = '2010';`  

| id | select_type | table  | partitions | type | possible_keys | key      | key_len | ref   | rows | filtered | Extra                 |
|----|-------------|--------|------------|------|---------------|----------|---------|-------|------|----------|-----------------------|
|  1 | SIMPLE      | papers | NULL       | ref  | year_idx      | year_idx | 1       | const |  162 |   100.00 | Using index condition |

Once again, only 162 rows from the table will be examined.  The Extra column indicates that the index will be used first to determine if a data table row has to be retrieved.  

---
### Putting extra data in an unclustered index ###
There's a **trick** that can be used for certain types of queries, which is to **put the extra data required by the query in the unclustered index**.  Then, the **query doesn't have to look at the data table records at all**, it can get the required information from the index.  

Replace the unclustered year_idx with an index on year and myid.  myid is being included in the index because it's what is required in the select.

`mysql> alter table Papers drop index year_idx`  
`mysql> alter table Papers add index year_myid_idx (year,myid)`  
`mysql> show indexes from Papers;`      

| Table  | Non_unique | Key_name      | Seq_in_index | Column_name | Collation | Cardinality | Sub_part | Packed | Null | Index_type | Comment | Index_comment |
|--------|------------|---------------|--------------|-------------|-----------|-------------|----------|--------|------|------------|---------|---------------|
| papers |          0 | PRIMARY       |            1 | narid       | A         |        1687 |     NULL | NULL   |      | BTREE      |         |               |
| papers |          1 | year_myid_idx |            1 | year        | A         |          13 |     NULL | NULL   |      | BTREE      |         |               |
| papers |          1 | year_myid_idx |            2 | myid        | A         |        1617 |     NULL | NULL   |      | BTREE      |         |               |

The table now has a **primary key on narid (clustered index)** and an **unclustered index on (year,myid)**.

`mysql> explain select myid from Papers where year = '2010';`    

| id | select_type | table  | partitions | type | possible_keys | key           | key_len | ref   | rows | filtered | Extra                    |
|----|-------------|--------|------------|------|---------------|---------------|---------|-------|------|----------|--------------------------|
|  1 | SIMPLE      | papers | NULL       | ref  | year_myid_idx | year_myid_idx | 1       | const |  162 |   100.00 | Using where; Using index |

Note the difference from the last time.  **Now Extra says "Using index", therefore, the data table is not used at all**.  This would make a **huge difference with a very large table**.  

<span style="color:red">**Adding required fields to an unclustered index can make it as efficient as a clustered index!**</span>

<span style="color:red">**However, don't duplicate the entire table in the index!  Indexes take up memory!**</span>

---
### EXPLAIN and JOINs ###

Now, look at EXPLAIN for a join query.  We start with no indexes in either table.

`explain select * from Papers join Reviewers using (narid)`


| id | select_type | table     | partitions | type | possible_keys | key  | key_len | ref  | rows | filtered | Extra                                              |
|----|-------------|-----------|------------|------|---------------|------|---------|------|------|----------|----------------------------------------------------|
|  1 | SIMPLE      | Papers    | NULL       | ALL  | NULL          | NULL | NULL    | NULL | 1687 |   100.00 | NULL                                               |
|  1 | SIMPLE      | Reviewers | NULL       | ALL  | NULL          | NULL | NULL    | NULL | 8123 |    10.00 | Using where; Using join buffer (Block Nested Loop) |

The explain shows the following:
- the first table examined is Papers
- **every row in Papers is read**
- **for each row in Papers**
    - **all the rows in Reviewers** are read to find the matching narids
    
<span style="color:red">**This means there are 1687 x 8123 = 13,703,501 rows read to answer this query!**</span>

<span style="color:red">**The query takes about 28 milliseconds (slow)!**</span>

### How to speed up the join ###

Consider what is happening:
- a row from Papers is examined and the narid is retrieved, call the value "x"
- the system now has to find any matching rows in Reviewers with narid = "x"
- if there's no index, the only possibility is a scan of all the rows of Reviewers
- instead of this, let's add an index to Reviewers that quickly finds the rows with narid = "x"

Create an index for narid in Reviewers  
`alter table Reviewers add index narid_idx (narid);`  
`show indexes from Reviewers;`  

| Table     | Non_unique | Key_name  | Seq_in_index | Column_name | Collation | Cardinality | Sub_part | Packed | Null | Index_type | Comment | Index_comment |
|-----------|------------|-----------|--------------|-------------|-----------|-------------|----------|--------|------|------------|---------|---------------|
| reviewers |          1 | narid_idx |            1 | narid       | A         |        1557 |     NULL | NULL   |      | BTREE      |         |               |

With the index, there is now a fast way to look up any narid value.

`explain select * from Papers join Reviewers using (narid)`

| id | select_type | table     | partitions | type | possible_keys | key       | key_len | ref                        | rows | filtered | Extra |
|----|-------------|-----------|------------|------|---------------|-----------|---------|----------------------------|------|----------|-------|
|  1 | SIMPLE      | Papers    | NULL       | ALL  | NULL          | NULL      | NULL    | NULL                       | 1687 |   100.00 | NULL  |
|  1 | SIMPLE      | Reviewers | NULL       | ref  | narid_idx     | narid_idx | 32      | editorsdb2020.Papers.narid |    5 |   100.00 | NULL  |

Look at what is different:
- now for each row of the Papers table, only 5 rows (on average) of the Reviewers table are examined
- the variable used for testing is narid from the Papers table (editorsdb2020.Papers.narid)

<span style="color:red">**Now there are 1687 x 5 = 8,435 rows read to answer this query!**</span>

<span style="color:red">**That's 13,703,501/8,435 = 1,624.6 fewer rows!**</span>

<span style="color:red">**The query takes about 1 milliseconds!**</span>

### Joins and Primary Keys ###

Normally, your tables will have primary keys and because of this, simple joins will be efficient.  

Consider these two tables with no indexes:

**Papers(narid, myid, year, title, author)**  
**Reviewers(narid,reviewerid,agree_decline,days,rating, year)**

We'll add the primary key to each table.

`alter table Papers add Primary Key (narid)`  
`alter table Reviewers add Primary Key (narid, reviewerid)`
`show indexes from Papers`
`show indexes from Reviewers`

The output of the show index commands are combined below.

| Table  | Non_unique | Key_name | Seq_in_index | Column_name | Collation | Cardinality | Sub_part | Packed | Null | Index_type | Comment | Index_comment |
|--------|------------|----------|--------------|-------------|-----------|-------------|----------|--------|------|------------|---------|---------------|
| papers |          0 | PRIMARY  |            1 | narid       | A         |        1687 |     NULL | NULL   |      | BTREE      |         |               |
| reviewers |          0 | PRIMARY  |            1 | narid       | A         |        1557 |     NULL | NULL   |      | BTREE      |         |               |
| reviewers |          0 | PRIMARY  |            2 | reviewerid  | A         |        8097 |     NULL | NULL   |      | BTREE      |         |               |


`explain select * from Papers join Reviewers using (narid)`

| id | select_type | table     | partitions | type | possible_keys | key       | key_len | ref                        | rows | filtered | Extra |
|----|-------------|-----------|------------|------|---------------|-----------|---------|----------------------------|------|----------|-------|
|  1 | SIMPLE      | Papers    | NULL       | ALL  | PRIMARY       | NULL    | NULL    | NULL                       | 1687 |   100.00 | NULL  |
|  1 | SIMPLE      | Reviewers | NULL       | ref  | PRIMARY       | PRIMARY | 32      | editorsdb2020.Papers.narid |    5 |   100.00 | NULL  |

Again, we get an efficient query, starting with Papers.

What would happen if the fields in the Primary Key for Reviewers were reversed?

`alter table Reviewers drop Primary Key`  
`alter table Reviewers add Primary Key (reviewerid, narid)`  
`show indexes from Papers`  
`show indexes from Reviewers`  

| Table  | Non_unique | Key_name | Seq_in_index | Column_name | Collation | Cardinality | Sub_part | Packed | Null | Index_type | Comment | Index_comment |
|--------|------------|----------|--------------|-------------|-----------|-------------|----------|--------|------|------------|---------|---------------|
| papers |          0 | PRIMARY  |            1 | narid       | A         |        1687 |     NULL | NULL   |      | BTREE      |         |               |
| reviewers |          0 | PRIMARY  |            1 | reviewerid  | A         |        4211 |     NULL | NULL   |      | BTREE      |         |               |
| reviewers |          0 | PRIMARY  |            2 | narid       | A         |        8097 |     NULL | NULL   |      | BTREE      |         |               |

**This creates a potential problem**  
If we select a row from Papers with an narid value = "x", there's no way to use the Primary Key in Reveiwers to quickly look up the correct Reviewer records.

To show this, the query has to be written in a special way:

`explain select * from Papers straight_join Reviewers on papers.narid=reviewers.narid;`

| id | select_type | table     | partitions | type | possible_keys | key       | key_len | ref                        | rows | filtered | Extra |
|----|-------------|-----------|------------|------|---------------|-----------|---------|----------------------------|------|----------|-------|
|  1 | SIMPLE      | Papers    | NULL       | ALL  | PRIMARY       | NULL | NULL    | NULL | 1687 |   100.00 | NULL                                               |
|  1 | SIMPLE      | Reviewers | NULL       | ALL  | NULL          | NULL | NULL    | NULL | 8097 |    10.00 | Using where; Using join buffer (Block Nested Loop) |

<span style="color:red">**We're back to an inefficient query, 13 million rows!**</span>

**However, the system is smart!**

**It can reorder the tables!**  That's why the query had to be written with `straight_join` to force it to use the order of the tables as written.  If we go back to using just `join`, then we get an efficient join.

`explain select * from Papers join Reviewers using(narid);`


| id | select_type | table     | partitions | type | possible_keys | key       | key_len | ref                        | rows | filtered | Extra |
|----|-------------|-----------|------------|------|---------------|-----------|---------|----------------------------|------|----------|-------|
|  1 | SIMPLE      | Reviewers | NULL       | ALL    | NULL          | NULL    | NULL    | NULL                          | 8097 |   100.00 | NULL  |
|  1 | SIMPLE      | Papers    | NULL       | eq_ref | PRIMARY       | PRIMARY | 32      | editorsdb2020.Reviewers.narid |    1 |   100.00 | NULL  |

The explain shows the following:  
- the first table examined is **Reviewers!**  
- **every row in Reviewers is read**  
- **for each row in Reviewers the narid value is examined** (editorsdb2020.Reviewers.narid)  
    - **find the single matching row in Papers**  
    
<span style="color:red">**The table order has been adjusted to produce an efficient query!**</span>

### Three table joins ###

Consider now these three tables and their normal primary keys.  **Note the order of fields for Reviewers**:

**Papers(narid, myid, year, title, author)**  
**Reviewers(narid,reviewerid,agree_decline,days,rating, year)**
**Uname(id, name)**

`alter table Papers add Primary Key (narid)`  
`alter table Reviewers add Primary Key (narid, reviewerid)`
`alter table Uname add Primary Key (id)`  
`show indexes from Papers`
`show indexes from Reviewers`
`show indexes from Uname`


| Table  | Non_unique | Key_name | Seq_in_index | Column_name | Collation | Cardinality | Sub_part | Packed | Null | Index_type | Comment | Index_comment |
|--------|------------|----------|--------------|-------------|-----------|-------------|----------|--------|------|------------|---------|---------------|
| papers |          0 | PRIMARY  |            1 | narid       | A         |        1687 |     NULL | NULL   |      | BTREE      |         |               |
| reviewers |          0 | PRIMARY  |            1 | narid       | A         |        1557 |     NULL | NULL   |      | BTREE      |         |               |
| reviewers |          0 | PRIMARY  |            2 | reviewerid  | A         |        8097 |     NULL | NULL   |      | BTREE      |         |               |
| uname |          0 | PRIMARY  |            1 | id          | A         |        5824 |     NULL | NULL   |      | BTREE      |         |               |

Consider the following three table query:

`select narid, title, name`  
`from Papers join Reviewers using (narid) join Uname on id = reviewerid`  

Let the three triangles below be the Primary Key indexes for the tables.  

<table style="width:100%">
  <tr>      </tr>
  <tr>
    <td><img src="https://upload.wikimedia.org/wikipedia/commons/7/71/Triangle_4.gif" width="150"></td>
    <td><img src="https://upload.wikimedia.org/wikipedia/commons/7/71/Triangle_4.gif" width="150"></td>    
    <td><img src="https://upload.wikimedia.org/wikipedia/commons/7/71/Triangle_4.gif" width="150"></td>  
  </tr>
  <tr>
    <td>Papers (narid)&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</td>
    <td>Reviewers (narid, reviewerid)&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</td>    
    <td>Uname (id)&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</td>  
  </tr>
 </table> 

Below are six different orders in which these tables can be combined (remember, the system can change the table order).  

**Determine which orders will work and which will be efficient.**  

An order works if the tables have a field in common to join on.  An order is efficient if a prior table can use an index in a subsequent table to look up the rows efficiently.
- P, R, U (Papers, Reviewers, Uname)
    - works
    - efficient
    - For each row of P 
        - get narid value "n"
        - use Reviewers index to find narid = "n" (because narid is first) and get reviewerid value "r"
        - use Uname index to find id = "r"
- P, U, R
    - doesn't work, nothing in common between P and U
- R, P, U
    
- R, U, P

- U, P, R

- U, R, P

Finally, this is the order the system chooses:

`explain`
`select narid, title, name`  
`from Papers join Reviewers using (narid) join Uname on id = reviewerid`  


| id | select_type | table     | partitions | type | possible_keys | key       | key_len | ref                        | rows | filtered | Extra |
|----|-------------|-----------|------------|------|---------------|-----------|---------|----------------------------|------|----------|-------|
|  1 | SIMPLE      | p     | NULL       | ALL    | PRIMARY       | NULL    | NULL    | NULL                       | 1687 |   100.00 | NULL        |
|  1 | SIMPLE      | r     | NULL       | ref    | PRIMARY       | PRIMARY | 32      | editorsdb2020.p.narid      |    5 |   100.00 | Using index |
|  1 | SIMPLE      | Uname | NULL       | eq_ref | PRIMARY       | PRIMARY | 4       | editorsdb2020.r.reviewerid |    1 |   100.00 | NULL        |


### Adding an extra index to a table ###

Finally, here is a select statement similar to the one above, but with a restriction to a single year.  However, it does no better than the query above.

`explain`  
`select narid, title, name, rating`  
`from Papers p join Reviewers using (narid) join UName on reviewerid=id`
`where p.year = '2012'`

| id | select_type | table     | partitions | type | possible_keys | key       | key_len | ref                        | rows | filtered | Extra |
|----|-------------|-----------|------------|------|---------------|-----------|---------|----------------------------|------|----------|-------|
|  1 | SIMPLE      | p     | NULL       | ALL    | PRIMARY       | NULL    | NULL    | NULL                       | 1687 |   100.00 | NULL        |
|  1 | SIMPLE      | r     | NULL       | ref    | PRIMARY       | PRIMARY | 32      | editorsdb2020.p.narid      |    5 |   100.00 | Using index |
|  1 | SIMPLE      | Uname | NULL       | eq_ref | PRIMARY       | PRIMARY | 4       | editorsdb2020.r.reviewerid |    1 |   100.00 | NULL        |


**Make suggestions for an index to add that would make the query more efficient and describe the resulting order of the table joins.** 



<!-- Add unclustered index (year, narid) to papers -->

<!--
`explain`  
`select narid, title, name, rating`  
`from Papers p join Reviewers using (narid) join UName on reviewerid=id`  
`where p.year = '2012'`


| id | select_type | table     | partitions | type | possible_keys | key       | key_len | ref                        | rows | filtered | Extra |
|----|-------------|-----------|------------|------|---------------|-----------|---------|----------------------------|------|----------|-------|
|  1 | SIMPLE      | p         | NULL       | ref    | PRIMARY,year_narid_idx | year_narid_idx | 1       | const                              |  141 |   100.00 | Using index condition |
|  1 | SIMPLE      | Reviewers | NULL       | ref    | PRIMARY                | PRIMARY        | 32      | editorsdb2020.p.narid              |    5 |   100.00 | NULL                  |
|  1 | SIMPLE      | UName     | NULL       | eq_ref | PRIMARY                | PRIMARY        | 4       | editorsdb2020.Reviewers.reviewerid |    1 |   100.00 | NULL                  |
-->